In [ ]:
import numpy as np
import xarray as xr
import pyproj
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import cartopy
import cartopy.crs as ccrs
import string

import sys
sys.path.append('../')

from plotfunctions import add_lonlat

np.seterr(all='ignore')
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
startyear = 2003
endyear   = 2008

In [ ]:
proj = ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0)#260-360)
axex = [245.8,251,-75.2,-74.4]
axex2 = [245.7,246.1,-75.11,-74.85]
axex3 = [246.8,247,-74.6,-74.3]
axex4 = [248.93,249.07,-75.35,-74.7]


cmap1 = mpl.colors.ListedColormap(plt.get_cmap('cmo.deep_r')(np.linspace(0,.4,255)))
cmap2 = 'cmo.rain_r'
cmap3 = mpl.colors.ListedColormap(plt.get_cmap('cmo.gray')(np.linspace(.4,.7,255)))

x0,x1,y0,y1 = 3400,3800,7700,8100

In [ ]:
vmax = 100
vmin = -10
linthresh=1
linscale=.2

vmax = 100
vmin = -10
linthresh = .3
linscale = .25


fracpos = (np.log10(vmax/linthresh)+linscale)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale)
#print(fracpos)
nneg = np.int_((1-fracpos)*256)

colors1 = plt.get_cmap('cmo.ice_r')(np.linspace(0,.9,nneg))
colors2 = plt.cm.inferno(np.linspace(0.1, 1, 256-nneg))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors)


In [ ]:
def makebackground(ax,axex):
    cmap = plt.get_cmap('ocean')
    ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
    ds = ds.isel(x=slice(x0,x1),y=slice(y0,y1))
    mask = xr.where(ds.mask==1,2,ds.mask)
    ds = add_lonlat(ds)
    
    ax.set_facecolor(".7")
    
    im1 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==0,ds.bed,np.nan),cmap=cmap1,shading='nearest',transform=ccrs.PlateCarree(),vmin=-1500,vmax=0)
    im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(mask==2,ds.bed,np.nan),cmap=cmap3,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)

    ax.contour(ds.lon,ds.lat,np.where(ds.mask==3,ds.thickness-ds.surface,np.nan),[450],colors='k',linewidths=.5,transform=ccrs.PlateCarree(),zorder=10)
    
    ax.set_extent(axex,crs=ccrs.PlateCarree())

In [ ]:
def add_zoom(ax,axexs):
    for axx in axexs:
        axins = inset_axes(ax,width="0%",height="0%",loc='lower right',borderpad=0,axes_class=cartopy.mpl.geoaxes.GeoAxes,axes_kwargs=dict(map_projection=proj))    
        axins.set_extent(axx,crs=ccrs.PlateCarree())
        ax.indicate_inset_zoom(axins, edgecolor="k",lw=1)
        
    ax.text(.73,.18,'KG',transform=ax.transAxes)
    ax.text(.25,.18,'DC',transform=ax.transAxes)
    ax.text(.37,.57,'CS',transform=ax.transAxes)

In [ ]:
def plotmelt(ax,lon,lat,melt,title):
    makebackground(ax,axex)
    ax.set_extent(axex,crs=ccrs.PlateCarree())
    im = ax.pcolormesh(lon,lat,melt,cmap=mymap,norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale),shading='nearest',transform=ccrs.PlateCarree())

    ax.set_title(title,loc='left',fontsize=10)
    
    add_zoom(ax,[axex2,axex3,axex4])
    
    return im

In [ ]:
plt.style.use('style_paper')

mpl.rc('figure.subplot',left=.01,right=.98,top=.95,bottom=.2,wspace=.05,hspace=.0)

fig = plt.figure(figsize=(7,3))

#Geometry

ax = fig.add_subplot(141,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))
makebackground(ax,axex)
ax.set_extent(axex,crs=ccrs.PlateCarree())
ax.set_title('a) Geometry',loc='left',fontsize=10)

ds = xr.open_dataset('../../../data/BedMachineAntarctica_2020-07-15_v02.nc')
ds = ds.isel(x=slice(x0,x1),y=slice(y0,y1))
ds = add_lonlat(ds)

im2 = ax.pcolormesh(ds.lon,ds.lat,np.where(ds.mask==3,ds.surface-ds.thickness,np.nan),cmap=cmap2,shading='nearest',transform=ccrs.PlateCarree(),vmin=-2000,vmax=0)

add_zoom(ax,[axex2,axex3,axex4])

#Obs
ax = fig.add_subplot(142,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))
ds = xr.open_dataset('../../../data/gourmelen/CrossonDotson.nc')
ds = add_lonlat(ds)
melt = ds.Band1
im = plotmelt(ax,ds.lon,ds.lat,melt,'b) Observations')

#MITgcm
ax = fig.add_subplot(143,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))

timep= slice(f"{startyear}-1-1",f"{endyear}-12-31")
ds = xr.open_dataset('../../../data/paulholland/melt.nc')
ds = ds.sel(LONGITUDE=slice(360-114.7,360-109),LATITUDE=slice(-75.3,-74.1),TIME=timep)
ds = ds.mean(dim='TIME')
lon   = (ds.LONGITUDE-360.).values
lat   = (ds.LATITUDE-.05).values
melt = ds.melt.values
im = plotmelt(ax,lon,lat,np.where(melt==0,np.nan,melt),'c) 3D model')

#LADDIE at 500m
ax = fig.add_subplot(144,projection=ccrs.SouthPolarStereo(true_scale_latitude=-75,central_longitude=0))

#ds = xr.open_dataset(f'../../results/spinup/CrossDots_0.5_mitgcm_{startyear}_{endyear}_100.nc')
ds = xr.open_dataset(f'../../results/CrossDots_0.5_tanh_Tdeep0.4_ztcl-500_050.nc')

plotmelt(ax,ds.lon,ds.lat,np.where(ds.tmask,ds.melt,np.nan),'d) LADDIE')
#print(ds.tmask.values.sum()*500**2)

ax = fig.add_subplot(111)
ax.set_visible(False)
cax = inset_axes(ax,width="24%",height="5%",loc='lower left',bbox_to_anchor=(0,-.08,1,1),bbox_transform=ax.transAxes,borderpad=0)
cb = plt.colorbar(im2,cax=cax,orientation='horizontal')
cb.set_label('Ice shelf draft $z_b$ [m]')

cax = inset_axes(ax,width="74%",height="5%",loc='lower left',bbox_to_anchor=(.26,-.08,1,1),bbox_transform=ax.transAxes,borderpad=0)
ticks = [-10,-3,-1,-.3,0,.3,1,3,10,30,100]
cb = plt.colorbar(im,cax=cax,orientation='horizontal',extend='both')
cb.set_ticks(ticks)
cb.set_ticklabels(ticks)
cb.set_label('Freezing / Melt rate $\dot{m}$ [m yr$^{-1}$]')

plt.savefig('../../figures/draftplot_CD_melt.png',dpi=450)